<a href="https://colab.research.google.com/github/jacomyma/mapping-controversies/blob/main/notebooks/Scrape_Reddit_posts_%26_comments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install praw

import requests
import json
import re
import time
import csv
import praw
import datetime as dt
import pandas as pd

In [ ]:
PUSHSHIFT_REDDIT_URL = "https://api.pullpush.io/reddit"
params = {
        "sort_type":"created_utc",
        "sort":"asc",
        "size": 1000,
        "subreddit": 'teachers',
        "q": 'ChatGPT'
        }

type = "submission"

r = requests.get(PUSHSHIFT_REDDIT_URL + "/" + type + "/search/", params=params, timeout=30)


if r.status_code == 200:
    response = json.loads(r.text)
    data = response['data']
    sorted_data_by_id = sorted(data, key=lambda x: int(x['id'],36))

sorted_data_by_id

In [ ]:
PUSHSHIFT_REDDIT_URL = "https://api.pullpush.io/reddit"

def fetchObjects(**kwargs):
    # Default paramaters for API query
    params = {
        "sort_type":"created_utc",
        "sort":"asc",
        "size": 1000,
        "subreddit": 'teachers',
        "q": 'artificial intelligence'
        }

    # Add additional paramaters based on function arguments
    for key,value in kwargs.items():
        params[key] = value

    # Print API query paramaters
    print(params)

    # Set the type variable based on function input
    # The type can be "comment" or "submission", default is "comment"
    type = "comment"
    if 'type' in kwargs and kwargs['type'].lower() == "submission":
        type = "submission"

    # Perform an API request
    r = requests.get(PUSHSHIFT_REDDIT_URL + "/" + type + "/search/", params=params, timeout=30)

    # Check the status code, if successful, process the data
    if r.status_code == 200:
        response = json.loads(r.text)
        data = response['data']
        sorted_data_by_id = sorted(data, key=lambda x: int(x['id'],36))
        return sorted_data_by_id

def extract_reddit_data(**kwargs):
    # Speficify the start timestamp
    max_created_utc = 1672563620  # 01/12/2021 @ 11:29pm - start with this
    max_id = 0

    # Open a file for JSON output
    with open('loop.csv','a') as f1:
        writer=csv.writer(f1, delimiter=',',lineterminator='\n',)

        # While loop for recursive function
        while 1:
            nothing_processed = True
            # Call the recursive function
            objects = fetchObjects(**kwargs,after=max_created_utc)

            # Loop the returned data, ordered by date
            for object in objects:
                id = int(object['id'],36)
                if id > max_id:
                    nothing_processed = False
                    created_utc = object['created_utc']
                    max_id = id
                    if created_utc > max_created_utc: max_created_utc = created_utc
                    # Output JSON data to the opened file
                    row = [object['id']]
                    writer.writerow(row)

            # Exit if nothing happened
            if nothing_processed: return
            max_created_utc -= 1

            # Sleep a little before the next recursive function call
            time.sleep(1)

# Start program by calling function with:
# 1) Subreddit specified
# 2) The type of data required (comment or submission)
extract_reddit_data(subreddit="teachers",type="submission")

In [ ]:
data = pd.read_csv("loop.csv",
                   sep=',',
                   names=["id"])


data

In [ ]:
reddit = praw.Reddit(client_id="hhxzoVnl0x0jlA",                      # client id
                     client_secret="wPJjQSLNIeMbtK3xilkrHdW3ftToCw",  # client secret
                     user_agent="scraper",                            # user agent name
                     username = "digitalscienceboi",                  # reddit username
                     password = "htHejLGDFLYVrX6",                    # reddit password
                     check_for_async=False)

post_dict = { "title":[],"score":[],"id":[], "url":[], "comms_num":[],"created":[],"body":[],"author":[]}
comment_dict = {"author":[],"score":[],"id":[],"created":[],"link_id":[],"body":[]}

for x in data['id']:
  submission = (praw.models.Submission(reddit, id = x))
  post_dict["title"].append(submission.title)
  post_dict["score"].append(submission.score)
  post_dict["id"].append(submission.id)
  post_dict["url"].append(submission.url)
  post_dict["comms_num"].append(submission.num_comments)
  post_dict["created"].append(submission.created)
  post_dict["body"].append(submission.selftext)
  post_dict["author"].append(submission.author)
  comments = submission.comments
  for comment in comments:
    comment_dict["author"].append(comment.author)
    comment_dict["score"].append(comment.score)
    comment_dict["id"].append(comment.id)
    comment_dict["created"].append(comment.created)
    comment_dict["link_id"].append(comment.link_id)
    comment_dict["body"].append(comment.body)


In [ ]:
comment_dict

In [ ]:
def get_date(created):
    return dt.datetime.fromtimestamp(created)

post_out = pd.DataFrame(post_dict)
comment_out = pd.DataFrame(comment_dict)

post_timestamp = post_out['created'].apply(get_date)
comment_timestamp = comment_out['created'].apply(get_date)

post_out = post_out.assign(timestamp = post_timestamp)
comment_out = comment_out.assign(timestamp = comment_timestamp)

post_out = post_out.sort_values(by=['score'], ascending=False)

post_out